In [1]:
from pytube import YouTube
from pydub import AudioSegment
import os
import speech_recognition as sr
import wave

In [2]:
def mp4_to_wav(input_file, output_file):
    audio = AudioSegment.from_file(input_file)
    
    audio = audio.set_channels(1)
    audio = audio.set_sample_width(2)
    audio = audio.set_frame_rate(16000)

    audio.export(output_file, format="wav")

    os.remove(input_file)


def verify_audio_format(wav_file):
    with wave.open(wav_file, 'rb') as file:
        if (wav_file.lower().endswith('.wav') and 
            file.getnchannels() == 1 and
            file.getsampwidth() == 2 and
            file.getframerate() == 16000):
            print('\nFile is in correct format.\n')
            return True
        else:
            print('\nFile is not in correct format.\n')
            return False


def download_youtube_audio(url, output_path):
    return YouTube(url).streams.filter(only_audio=True).first().download(
        output_path=output_path
    )


def transcribe_audio(wav_file):
    if not verify_audio_format(wav_file):
        raise Exception('Cannot transcribe audio with invalid file format.')
        
    speech_recognizer = sr.Recognizer()

    with sr.AudioFile(wav_file) as file:
        audio = speech_recognizer.record(file)

    try:
        text = speech_recognizer.recognize_sphinx(audio)
        return text
    except Exception as e:
        print('Unable to transcribe audio.')
        print(e)

In [3]:
# url = 'https://www.youtube.com/watch?v=pat2-f0BXCg'
url = 'https://www.youtube.com/watch?v=ry9SYnV3svc'

# create the output directory to house audio file downloads
output_path = 'Output/'
os.makedirs(output_path, exist_ok=True)

# convert and download an audio file (in .mp4 format) from a youtube url
mp4_file = download_youtube_audio(url, output_path)
print('DOWNLOADED FILE:', mp4_file)

# get the name of the file without file path or file extension
file_name, _ = os.path.splitext(os.path.basename(mp4_file))

# set the full path of the .wav file for the audio
wav_file = os.path.join(output_path, file_name + '.wav')
print('OUTPUT FILE:', wav_file)

# convert the file from .mp4 to .wav
mp4_to_wav(mp4_file, wav_file)
del mp4_file
print('CONVERTED FILE:', wav_file)

# transcribe the .wav audio file into text
transcription = transcribe_audio(wav_file)
print(transcription)

DOWNLOADED FILE: /Users/mattkarenbauer/Coding/Data Science/LyricalSentimentAnalysis/Output/Small Talk  Everyday English.mp4
OUTPUT FILE: Output/Small Talk  Everyday English.wav
CONVERTED FILE: Output/Small Talk  Everyday English.wav

File is in correct format.

but the so what's new mark how was your new job going to be honest i can't complain i really love the company that i am working for my co workers are all really friendly and helpful they really helped me feel welcome if they're really energetic and find on this here in my boss beutel areas and he's really flexible really how so he allows me to come in when i want and make my own al words i can also leave early if i start early there is no real dress code either i can wear jeans and a t. shirt if i wanted to i didn't even wear shorts in the summer while it sounds you really cool mikey it's the end wearing a suit every day which do you prefer working late or finish in the early i prefer finishing early i really enjoy the morning i